In [2]:
import pandas as pd


def read_builds_info_change_ids(file_name):
    with open(file_name, "r") as json_file:
        loaded_dict = pd.read_json(json_file)
        converted_dict = {
            tuple(
                key.replace("(", "").replace(")", "").replace(" ", "").split(",")
            ): value
            for key, value in loaded_dict.items()
        }

    change_ids = set()
    for key, value in converted_dict.items():
        change_ids.add(key[0].replace("'", ""))

    return change_ids

In [110]:
from pymongo import MongoClient
import json
import pandas as pd

conn = MongoClient("127.0.0.1", 27017)
db = conn["recheck_prediction"]
collection = db["patch_and_user"]

pipeline = [{"$match": {"branch": "master"}}]

result = list(collection.aggregate(pipeline))

change_ids = read_builds_info_change_ids("../../data/data.json")

print("L")
print(len(change_ids))

pipeline_duplicates = [
    {"$group": {"_id": "$change_id", "count": {"$sum": 1}}},
    {"$match": {"count": {"$gt": 1}}},
    {
        "$lookup": {
            "from": "patch_and_user",
            "localField": "_id",
            "foreignField": "change_id",
            "as": "matches",
        }
    },
    {"$unwind": "$matches"},
    {"$match": {"matches.branch": "master"}},
]

# Pipeline to find unique change_id values
pipeline_unique = [
    {"$group": {"_id": "$change_id", "count": {"$sum": 1}}},
    {"$match": {"count": 1}},
    {
        "$lookup": {
            "from": "patch_and_user",
            "localField": "_id",
            "foreignField": "change_id",
            "as": "matches",
        }
    },
]

# Execute the aggregation pipelines
result_duplicates = list(collection.aggregate(pipeline_duplicates))
result_unique = list(collection.aggregate(pipeline_unique))

# result_duplicates.extend(result_unique)

L
22671


In [107]:
pipeline_unique_change_id = [
    {"$group": {"_id": "$change_id", "count": {"$sum": 1}}},
    {"$match": {"count": 1}},
]

unique_change_ids = [
    item["_id"] for item in collection.aggregate(pipeline_unique_change_id)
]

# Filter out documents where change_id is unique and branch is not master
filtered_documents = list(
    collection.find(
        {"change_id": {"$in": unique_change_ids}, "branch": {"$ne": "master"}}
    )
)

In [108]:
print(len(filtered_documents))

252


[{'_id': ObjectId('657b401808d602155ef17a04'),
  'id': 'openstack%2Fcinder~stable%2Ftrain~I3f2ec1ddf1867261253190953f7a65fff22c7404',
  'project': 'openstack/cinder',
  'branch': 'stable/train',
  'topic': 'bp/vxflexos-replication-support',
  'hashtags': [],
  'change_id': 'I3f2ec1ddf1867261253190953f7a65fff22c7404',
  'subject': 'Add OpenStack volume replication v2.1 support in VxFlex OS driver',
  'status': 'ABANDONED',
  'created': '2020-04-28 07:45:09.000000000',
  'updated': '2020-07-01 16:02:09.000000000',
  'insertions': 799,
  'deletions': 50,
  'total_comment_count': 0,
  'unresolved_comment_count': 0,
  'has_review_started': True,
  'meta_rev_id': 'a2aeab8f3578a6fbd91e00f9fdcd94a8a38ff6bf',
  '_number': 723826,
  'owner': {'_account_id': 31016,
   'name': 'Ivan Pchelintsev',
   'email': 'i.pchelintsev@yadro.com',
   'username': 'pcheli'},
  'current_revision': '6064ec5af03525f2f6643b4b577fb7dc4d9e3034',
  'revisions': {'d9df2185247c5f32396e7d79eba981577a820a32': {'kind': 'REW

In [109]:
import time
import requests


for value in filtered_documents:
    response = requests.get(
        f"https://review.openstack.org/changes/?q=change:{value['change_id']}&o=ALL_REVISIONS&o=ALL_FILES&o=DETAILED_ACCOUNTS"
    )
    json_content = json.loads(response.content[4:])
    for res in json_content:
        collection.insert_one(res)
    time.sleep(2)

In [84]:
result_unique[0]

{'_id': 'If54d634abfd345c1e672d31cb11de14260b34cc0',
 'count': 1,
 'matches': [{'_id': ObjectId('657b8e353893e7488f449bfa'),
   'id': 'openstack%2Fheat~master~If54d634abfd345c1e672d31cb11de14260b34cc0',
   'project': 'openstack/heat',
   'branch': 'master',
   'topic': 'drop-six-and-py27-support',
   'hashtags': [],
   'change_id': 'If54d634abfd345c1e672d31cb11de14260b34cc0',
   'subject': 'Remove six and python 2.7 full support',
   'status': 'MERGED',
   'created': '2019-11-20 18:38:45.000000000',
   'updated': '2020-04-26 16:24:14.000000000',
   'submitted': '2020-04-26 16:22:27.000000000',
   'submitter': {'_account_id': 22348,
    'name': 'Zuul',
    'username': 'zuul',
    'tags': ['SERVICE_USER']},
   'insertions': 21,
   'deletions': 37,
   'total_comment_count': 0,
   'unresolved_comment_count': 0,
   'has_review_started': True,
   'submission_id': '695264-1587918147702-174f2d3f',
   'meta_rev_id': '68fcf1b400d9b4151a6f8bc29dab23e0fe18463a',
   '_number': 695264,
   'owner': {

In [38]:
print(len(result_duplicates))

1275


In [25]:
# Function to find non-unique change_ids
def find_non_unique_change_ids(arr):
    counts = {}
    non_unique_objects = []

    # Count occurrences of change_id
    for obj in arr:
        # print(obj.keys())
        counts[obj["_id"]] = counts.get(obj["_id"], 0) + 1

    # Find objects with non-unique change_ids
    for obj in arr:
        if counts[obj["_id"]] > 1 and obj not in non_unique_objects:
            non_unique_objects.append(obj)

    return non_unique_objects

In [111]:
# # Find and print objects with non-unique change_id
# print(result_duplicates[:1])
# non_unique_objects = find_non_unique_change_ids(result_duplicates)
# print("Objects with non-unique change_id:")

# projects = set()
# for obj in non_unique_objects:
#     # print(obj)
#     projects.add(obj["matches"]["project"])
# print(projects)
# print(len(projects))
grouped_duplicates = {}
for obj in result_duplicates:
    _id: str = obj["_id"]
    if _id not in grouped_duplicates:
        grouped_duplicates[_id] = []
    grouped_duplicates[_id].append((obj, len(obj["matches"]["revisions"])))

for group_id, group in grouped_duplicates.items():
    group.sort(key=lambda a: a[1], reverse=True)


# I need to leave the duplicate where the revisions length is max among duplicates

In [112]:
required_duplicates = []
for group_id, group in grouped_duplicates.items():
    required_duplicates.append(group[0][0])

In [113]:
print(len(required_duplicates) + len(result_unique))

22671


In [51]:
print(len(change_ids))

22671


In [114]:
required_data = required_duplicates + result_unique

In [54]:
required_data[:1]

[{'_id': 'Ia91b63c0676f42ad8a7a0d16e6870bafc2ee7675',
  'count': 3,
  'matches': {'_id': ObjectId('657b87483893e7488f449873'),
   'id': 'openstack%2Fnova~master~Ia91b63c0676f42ad8a7a0d16e6870bafc2ee7675',
   'project': 'openstack/nova',
   'branch': 'master',
   'topic': 'bug/1888022',
   'hashtags': [],
   'change_id': 'Ia91b63c0676f42ad8a7a0d16e6870bafc2ee7675',
   'subject': 'Detach is broken for multi-attached fs-based volumes',
   'status': 'MERGED',
   'created': '2020-07-17 20:41:18.000000000',
   'updated': '2020-08-13 02:51:32.000000000',
   'submitted': '2020-08-13 02:49:59.000000000',
   'submitter': {'_account_id': 22348,
    'name': 'Zuul',
    'username': 'zuul',
    'tags': ['SERVICE_USER']},
   'insertions': 63,
   'deletions': 30,
   'total_comment_count': 24,
   'unresolved_comment_count': 0,
   'has_review_started': True,
   'submission_id': '741712-1597287000193-bd91aa04',
   'meta_rev_id': 'a7786e4e13f12c5a1eb4972002e868e496e7e22d',
   '_number': 741712,
   'owner'

In [115]:
# get data by change id and get inserted, deleted by revision number

df = pd.read_csv(
    "../../data/features/features_combined.csv",
)


# df = df.to_dict()
def calculate_inserted_and_deleted_lines(row):
    # id = row[0]
    # try:
    change_id, revision_number, rerun_number, *_ = eval(row["id"])
    # required_data.index()
    # python_indices = []
    # for item in required_data:
    #     if item["_id"] == change_id:
    #         python_indices.append(item)
    python_indices = [item for item in required_data if item["_id"] == change_id][0]
    revision = [
        # item
        # for (index, item) in enumerate(
        #     python_indices["matches"][0]["revisions"].values()
        # )
        # if item["_number"] == revision_number
    ]
    matches_indices = python_indices["matches"]
    if isinstance(matches_indices, dict):
        check_index = matches_indices
    else:
        assert isinstance(matches_indices, list)
        assert len(matches_indices) > 0
        check_index = matches_indices[0]
    for item in check_index["revisions"].values():
        if item["_number"] == revision_number:
            revision.append(item)

    lines_inserted = 0
    lines_deleted = 0
    # if len()
    for value in revision[0]["files"].values():
        lines_inserted += value.get("lines_inserted", 0)
        lines_deleted += value.get("lines_deleted", 0)

    row["lines_inserted"] = lines_inserted
    row["lines_deleted"] = lines_deleted
    # except:
    #     print('Er')
    return row


# part_df = df.copy()[:10]
df_with_patch_info = df.apply(calculate_inserted_and_deleted_lines, axis=1)
df_with_patch_info[:10]

,id,user_success_ratio,bots_success_ratio,bots_flip_success_ratio,jobs_success_ratio,label,lines_inserted,lines_deleted
0,"('I3fafc4ef1cf56cb8f1cddc1a003a755e3f93c75c', ...",0.243421,0.393384,-1.000000,0.527473,0,57,2
1,"('I3fafc4ef1cf56cb8f1cddc1a003a755e3f93c75c', ...",0.242358,0.383812,-1.000000,0.602600,0,57,2
2,"('I3fafc4ef1cf56cb8f1cddc1a003a755e3f93c75c', ...",0.240781,0.383613,-1.000000,0.603380,0,57,2
3,"('I3fafc4ef1cf56cb8f1cddc1a003a755e3f93c75c', ...",0.333333,0.383444,-1.000000,0.604155,0,57,2
4,"('I3fafc4ef1cf56cb8f1cddc1a003a755e3f93c75c', ...",0.250000,0.384151,-1.000000,0.601699,0,57,2
5,"('I3fafc4ef1cf56cb8f1cddc1a003a755e3f93c75c', ...",0.166667,0.383688,-1.000000,0.596215,0,57,2
6,"('I3fb385eb0d3d7fa5e3747762f9b923e99ec621ca', ...",0.375000,0.321192,-1.000000,0.594591,0,170,22
7,"('I3fb39589ac85c6c7d74ad61c86a971130823af08', ...",0.042289,0.407377,0.811798,0.400000,0,10,0
8,"('I3fb70478eb3eac31955b75ed591eaa309482a035', ...",0.134454,0.319969,0.562849,0.131514,1,55,29
9,"('I3fb719c2b8c92d2aebb76698674ada6b69ad2c8f', ...",0.164062,0.316558,0.558988,0.287379,1,1,0


In [116]:
df_with_patch_info.to_csv("../../data/features/features_with_patch_info.csv")

In [126]:
# Normalize data

from sklearn import preprocessing
import numpy as np

df_with_patch_info["lines_inserted"] = preprocessing.normalize(
    [list(df_with_patch_info["lines_inserted"])]
).flatten()
df_with_patch_info["lines_deleted"] = preprocessing.normalize(
    [list(df_with_patch_info["lines_deleted"])]
).flatten()
# print(normalized_arr)

In [127]:
df_with_patch_info.to_csv("../../data/features/features_with_patch_info_normalized.csv")

In [140]:
def calculate_number_of_unique_files(row):
    change_id, revision_number, rerun_number, *_ = eval(row["id"])
    python_indices = [item for item in required_data if item["_id"] == change_id][0]
    revision = []
    matches_indices = python_indices["matches"]
    if isinstance(matches_indices, dict):
        check_index = matches_indices
    else:
        assert isinstance(matches_indices, list)
        assert len(matches_indices) > 0
        check_index = matches_indices[0]
    for item in check_index["revisions"].values():
        if item["_number"] == revision_number:
            revision.append(item)

    row["number_of_unique_files_touched"] = len(revision[0]["files"])
    return row

In [142]:
import pandas as pd

print(df_with_patch_info.keys())

df_with_patch_info = pd.read_csv("../../data/features/features_with_patch_info.csv")

df_with_patch_info = df_with_patch_info.apply(calculate_number_of_unique_files, axis=1)

Index(['id', 'user_success_ratio', 'bots_success_ratio',
       'bots_flip_success_ratio', 'jobs_success_ratio', 'label',
       'number_of_unique_files_touched'],
      dtype='object')


In [139]:
# part_df = df.copy()[:10]
df_with_patch_info = df.apply(calculate_number_of_unique_files, axis=1)

KeyboardInterrupt: 

In [129]:
df_with_patch_info["number_of_unique_files_touched"] = preprocessing.normalize(
    [list(df_with_patch_info["number_of_unique_files_touched"])]
).flatten()

In [138]:
df_with_patch_info.to_csv(
    "../../data/features/features_with_patch_info_normalized_files_1.csv"
)

In [134]:
import pandas as pd

print(df_with_patch_info.keys())

df_path_info = pd.read_csv("../../data/features/features_with_patch_info.csv")
df_path_info["number_of_unique_files_touched"] = df_with_patch_info[
    "number_of_unique_files_touched"
]
df_path_info.to_csv("../../data/features/features_with_patch_info_normalized_files.csv")

Index(['id', 'user_success_ratio', 'bots_success_ratio',
       'bots_flip_success_ratio', 'jobs_success_ratio', 'label',
       'number_of_unique_files_touched'],
      dtype='object')


In [137]:
print(len(df_path_info[df_path_info["label"] == 1]))
print(len(df_path_info))

18016
64482
